# Motorcycle sales analysis

In [17]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",100)

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error,r2_score

from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()

import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("sales_data.csv") #https://www.kaggle.com/datasets/devijeganath/motorcycle-sales-analysis

### EDA

In [16]:
df

,warehouse,client_type,product_line,quantity,unit_price,total,payment,day,month,year
0,0,0,4,8,16.85,134.83,0,1,6,2021
1,1,0,0,9,19.29,173.61,2,1,6,2021
2,1,0,1,8,32.93,263.45,0,1,6,2021
3,1,1,3,16,37.84,605.44,1,1,6,2021
4,0,0,5,2,60.48,120.96,0,1,6,2021
...,...,...,...,...,...,...,...,...,...,...
995,0,0,2,9,32.87,295.83,0,28,8,2021
996,2,1,0,32,10.03,320.96,1,28,8,2021
997,2,1,2,12,32.80,393.64,1,28,8,2021
998,1,0,3,5,48.25,241.23,2,28,8,2021


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   warehouse     1000 non-null   int32  
 1   client_type   1000 non-null   int32  
 2   product_line  1000 non-null   int32  
 3   quantity      1000 non-null   int64  
 4   unit_price    1000 non-null   float64
 5   total         1000 non-null   float64
 6   payment       1000 non-null   int32  
 7   day           1000 non-null   int32  
 8   month         1000 non-null   int32  
 9   year          1000 non-null   int32  
dtypes: float64(2), int32(7), int64(1)
memory usage: 50.9 KB


In [5]:
df.isnull().sum() #We examine the empty lines.

date            0
warehouse       0
client_type     0
product_line    0
quantity        0
unit_price      0
total           0
payment         0
dtype: int64

In [7]:
df["product_line"].value_counts()

Breaking system          230
Suspension & traction    228
Electrical system        193
Frame & body             166
Miscellaneous            122
Engine                    61
Name: product_line, dtype: int64

In [8]:
df["client_type"].value_counts()

Retail       775
Wholesale    225
Name: client_type, dtype: int64

In [9]:
df["warehouse"].value_counts()

Central    480
North      340
West       180
Name: warehouse, dtype: int64

In [10]:
df["payment"].value_counts()

Credit card    659
Transfer       225
Cash           116
Name: payment, dtype: int64

### Feature Engineering

In [11]:
df["product_line"]=df["product_line"].map({"Breaking system":0,"Suspension & traction":1,"Electrical system":2,"Frame & body":3,"Miscellaneous":4,"Engine":5})
#We replace the data in product_line with 0, 1, 2, 3, 4 and 5.
df['product_line']=df['product_line'].astype(int) #We change the data type of product_line to integer.

df["client_type"]=df["client_type"].map({"Retail":0,"Wholesale":1})
df['client_type']=df['client_type'].astype(int)

df["warehouse"]=df["warehouse"].map({"Central":0,"North":1,"West":2})
df['warehouse']=df['warehouse'].astype(int)

df["payment"]=df["payment"].map({"Credit card":0,"Transfer":1,"Cash":2})
df['payment']=df['payment'].astype(int)

In [12]:
df["date"]=pd.to_datetime(df["date"])
df["day"]=(df["date"]).dt.day
df["month"]=(df["date"]).dt.month
df["year"]=(df["date"]).dt.year
del df["date"]
df['day']=df['day'].astype(int)
df['month']=df['month'].astype(int)
df['year']=df['year'].astype(int)
#We divide date into day, month and year.

In [20]:
abs(df.corr()["total"].sort_values(ascending=False)) #We look at their correlations.

total           1.000000
quantity        0.870207
client_type     0.656483
unit_price      0.372942
payment         0.275685
product_line    0.217674
month           0.039390
warehouse       0.031947
day             0.061492
year                 NaN
Name: total, dtype: float64

### Regression

In [23]:
x,y=df.drop(["total","month","warehouse","day","year"],axis=1),df[["total"]]
x=scaler.fit_transform(x)
x.shape

(1000, 5)

In [25]:
def algo_test(x,y):
    L = LinearRegression()
    E = ElasticNet()
    R = Ridge()
    Lass = Lasso()
    ETR=ExtraTreeRegressor()
    GBR=GradientBoostingRegressor()
    XGBC= XGBRegressor()
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=13)
    algos = [L,E,R,Lass,ETR,GBR,XGBC]
    algo_names = ['Linear','ElasticNet','Ridge','Lasso','Extra Tree','Gradient Boosting','XGradientBooting']
    r_squared = []
    rmse = []
    mae = []
    result = pd.DataFrame(columns = ['R_Squared','RMSE','MAE'],index = algo_names)
    for algo in algos:
        algo.fit(x_train,y_train)    
        r_squared.append(r2_score(y_test,algo.predict(x_test)))
        rmse.append(mean_squared_error(y_test, algo.predict(x_test))**.5)
        mae.append(mean_absolute_error(y_test, algo.predict(x_test)))
    result.R_Squared = r_squared
    result.RMSE = rmse
    result.MAE= mae
    return result.sort_values('R_Squared', ascending=False)

In [26]:
algo_test(x,y)

,R_Squared,RMSE,MAE
Gradient Boosting,0.993412,29.562517,13.045627
XGradientBooting,0.987806,40.219284,12.410100
Extra Tree,0.987079,41.401250,16.976450
Lasso,0.859722,136.411966,72.963078
Linear,0.859273,136.630471,75.497196
Ridge,0.859268,136.632623,73.758383
ElasticNet,0.327339,298.714716,187.590509


In [27]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=42)

In [28]:
model=Sequential()
model.add(Dense(60,activation="relu"))
model.add(Dense(60,activation="relu"))
model.add(Dense(60,activation="relu"))
model.add(Dense(60,activation="relu"))
model.add(Dense(60,activation="relu"))
model.add(Dense(1))
model.compile(optimizer="adam",loss="mse")

In [29]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=10,epochs=500)
model.summary()

Epoch 1/500
80/80 [==============================] - 1s 3ms/step - loss: 189875.0938 - val_loss: 46262.6836
Epoch 2/500
80/80 [==============================] - 0s 1ms/step - loss: 58869.7500 - val_loss: 30124.2129
Epoch 3/500
80/80 [==============================] - 0s 1ms/step - loss: 39131.4414 - val_loss: 17285.6484
Epoch 4/500
80/80 [==============================] - 0s 1ms/step - loss: 25285.1367 - val_loss: 12929.2051
Epoch 5/500
80/80 [==============================] - 0s 1ms/step - loss: 17089.8066 - val_loss: 8010.4800
Epoch 6/500
80/80 [==============================] - 0s 2ms/step - loss: 11642.1904 - val_loss: 3999.7190
Epoch 7/500
80/80 [==============================] - 0s 1ms/step - loss: 7866.6968 - val_loss: 2671.1572
Epoch 8/500
80/80 [==============================] - 0s 1ms/step - loss: 5059.9126 - val_loss: 1627.8253
Epoch 9/500
80/80 [==============================] - 0s 1ms/step - loss: 3936.0315 - val_loss: 1552.4944
Epoch 10/500
80/80 [========================

80/80 [==============================] - 0s 1ms/step - loss: 54.2712 - val_loss: 48.9880
Epoch 159/500
80/80 [==============================] - 0s 1ms/step - loss: 638.0349 - val_loss: 3716.3738
Epoch 160/500
80/80 [==============================] - 0s 1ms/step - loss: 1304.8661 - val_loss: 204.9381
Epoch 161/500
80/80 [==============================] - 0s 1ms/step - loss: 154.8501 - val_loss: 127.8288
Epoch 162/500
80/80 [==============================] - 0s 1ms/step - loss: 67.9158 - val_loss: 62.9726
Epoch 163/500
80/80 [==============================] - 0s 1ms/step - loss: 35.5246 - val_loss: 52.5555
Epoch 164/500
80/80 [==============================] - 0s 1ms/step - loss: 29.8518 - val_loss: 50.6256
Epoch 165/500
80/80 [==============================] - 0s 1ms/step - loss: 33.5417 - val_loss: 42.6878
Epoch 166/500
80/80 [==============================] - 0s 1ms/step - loss: 32.6180 - val_loss: 49.4706
Epoch 167/500
80/80 [==============================] - 0s 1ms/step - loss: 26.5

80/80 [==============================] - 0s 1ms/step - loss: 96.4329 - val_loss: 35.4650
Epoch 317/500
80/80 [==============================] - 0s 1ms/step - loss: 26.8116 - val_loss: 33.2882
Epoch 318/500
80/80 [==============================] - 0s 1ms/step - loss: 29.0963 - val_loss: 29.4147
Epoch 319/500
80/80 [==============================] - 0s 1ms/step - loss: 21.5078 - val_loss: 44.6807
Epoch 320/500
80/80 [==============================] - 0s 1ms/step - loss: 84.4727 - val_loss: 28.5286
Epoch 321/500
80/80 [==============================] - 0s 1ms/step - loss: 60.3339 - val_loss: 98.1976
Epoch 322/500
80/80 [==============================] - 0s 1ms/step - loss: 32.4659 - val_loss: 25.8570
Epoch 323/500
80/80 [==============================] - 0s 1ms/step - loss: 25.6444 - val_loss: 32.9799
Epoch 324/500
80/80 [==============================] - 0s 1ms/step - loss: 98.0271 - val_loss: 186.5295
Epoch 325/500
80/80 [==============================] - 0s 1ms/step - loss: 1329.4175 -

80/80 [==============================] - 0s 1ms/step - loss: 239.9498 - val_loss: 137.0868
Epoch 475/500
80/80 [==============================] - 0s 1ms/step - loss: 61.7409 - val_loss: 27.9041
Epoch 476/500
80/80 [==============================] - 0s 1ms/step - loss: 23.4499 - val_loss: 32.1699
Epoch 477/500
80/80 [==============================] - 0s 1ms/step - loss: 13.2082 - val_loss: 22.6111
Epoch 478/500
80/80 [==============================] - 0s 1ms/step - loss: 32.0778 - val_loss: 39.2139
Epoch 479/500
80/80 [==============================] - 0s 1ms/step - loss: 18.6349 - val_loss: 21.2550
Epoch 480/500
80/80 [==============================] - 0s 1ms/step - loss: 12.6775 - val_loss: 23.9599
Epoch 481/500
80/80 [==============================] - 0s 1ms/step - loss: 38.3692 - val_loss: 26.0089
Epoch 482/500
80/80 [==============================] - 0s 1ms/step - loss: 18.1992 - val_loss: 21.1226
Epoch 483/500
80/80 [==============================] - 0s 1ms/step - loss: 17.1899 - 

In [30]:
tahmin=model.predict(x_test)

7/7 [==============================] - 0s 831us/step


In [31]:
r2_score(tahmin,y_test)

0.9992183336843911

In [32]:
(mean_squared_error(tahmin,y_test))**0.5

7.017635927850327